In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import ast
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv


In [2]:
movies = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')
credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies = movies.merge(credits,on='title')


movies_feature_set = movies[['id','title','keywords','genres','cast','popularity','vote_average','vote_count']]
movies_feature_set.dropna(inplace=True)
movies_feature_set.head()


/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,id,title,keywords,genres,cast,popularity,vote_average,vote_count
0,19995,Avatar,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...",150.437577,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...",139.082615,6.9,4500
2,206647,Spectre,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",107.376788,6.3,4466
3,49026,The Dark Knight Rises,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",112.312950,7.6,9106
4,49529,John Carter,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...",43.926995,6.1,2124


In [3]:


def convert(text):
    arr = []
    for i in ast.literal_eval(text):
        arr.append(i['name'])
    
    return arr

movies_feature_set['genres'] = movies['genres'].apply(convert)


def convert_keyword_top3(text):
    arr= []
    count = 0
    for i in ast.literal_eval(text):
        if count<3:
            arr.append(i['name'])
            count+=1
    return arr   

movies_feature_set['keywords'] = movies['keywords'].apply(convert_keyword_top3)

def convert_cast_top2(text):
    arr= []
    count = 0
    for i in ast.literal_eval(text):
        if count<2:
            arr.append(i['name'])
            count+=1
    return arr        
movies_feature_set['cast'] = movies['cast'].apply(convert_cast_top2)
movies_feature_set['tags'] = movies_feature_set['keywords'] + movies_feature_set['genres'] + movies_feature_set['cast']
new_df = movies_feature_set[['id','title','tags']]
new_df.head(5)




/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,id,title,tags
0,19995,Avatar,"[culture clash, future, space war, Action, Adv..."
1,285,Pirates of the Caribbean: At World's End,"[ocean, drug abuse, exotic island, Adventure, ..."
2,206647,Spectre,"[spy, based on novel, secret agent, Action, Ad..."
3,49026,The Dark Knight Rises,"[dc comics, crime fighter, terrorist, Action, ..."
4,49529,John Carter,"[based on novel, mars, medallion, Action, Adve..."


In [4]:
user_movie_ratings = pd.pivot_table(movies_feature_set, index='id', columns= 'title', values='vote_average')
user_movie_ratings

title,#Horror,(500) Days of Summer,10 Cloverfield Lane,10 Days in a Madhouse,10 Things I Hate About You,102 Dalmatians,10th & Wolf,11:14,12 Angry Men,12 Rounds,...,Zoolander,Zoolander 2,Zoom,Zulu,[REC],[REC]²,eXistenZ,xXx,xXx: State of the Union,Æon Flux
id,,,,,,,,,,,,,,,,,,,,,
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
426469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
X = movies_feature_set.iloc[:, :-1]
y = movies_feature_set.iloc[:, -1]
 
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

X_train

,id,title,keywords,genres,cast,popularity,vote_average,vote_count
1907,15487,The Greatest Game Ever Played,[sport],"[Drama, Family]","[Shia LaBeouf, Stephen Dillane]",7.538513,6.9,110
3157,31174,Richard III,"[england, shakespeare, kidnapping]","[Drama, War]","[Ian McKellen, Annette Bening]",5.107235,6.9,49
4093,20862,Smoke Signals,[],"[Drama, Comedy]","[Adam Beach, Evan Adams]",2.304552,6.6,24
1433,11004,Wonder Boys,"[adultery, robbery, based on novel]","[Comedy, Drama]","[Michael Douglas, Tobey Maguire]",9.011729,6.9,161
2513,18777,Slow Burn,[],"[Mystery, Crime, Drama, Thriller]","[Ray Liotta, LL Cool J]",6.630856,5.5,16
...,...,...,...,...,...,...,...,...
1033,9095,Mary Reilly,"[servant, monster, laboratory]","[Drama, Horror, Thriller, Romance]","[Julia Roberts, John Malkovich]",6.067212,5.7,74
3264,15049,Bon Cop Bad Cop,"[canada, police, murder]","[Action, Comedy, Crime]","[Patrick Huard, Colm Feore]",5.069095,6.5,42
1653,11208,Wicker Park,"[love of one's life, leave, look-alike]","[Drama, Mystery, Romance, Thriller]","[Josh Hartnett, Rose Byrne]",9.301562,6.7,157
2607,17644,Barney's Great Adventure,[],[Family],[],0.722911,2.9,8
